# Глубинное обучение для текстовых данных, ФКН ВШЭ

## Домашнее задание 2: Рекуррентные нейронные сети

### Оценивание и штрафы

Максимально допустимая оценка за работу — __10 (+3) баллов__. Сдавать задание после указанного срока сдачи нельзя.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Весь код должен быть написан самостоятельно. Чужим кодом для пользоваться запрещается даже с указанием ссылки на источник. В разумных рамках, конечно. Взять пару очевидных строчек кода для реализации какого-то небольшого функционала можно.

Неэффективная реализация кода может негативно отразиться на оценке. Также оценка может быть снижена за плохо читаемый код и плохо оформленные графики. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

__Мягкий дедлайн: 5.10.25 23:59__   
__Жесткий дедлайн: 8.10.25 23:59__

### О задании

В этом задании вам предстоит самостоятельно реализовать модель LSTM для решения задачи классификации с пересекающимися классами (multi-label classification). Это вид классификации, в которой каждый объект может относиться одновременно к нескольким классам. Такая задача часто возникает при классификации фильмов по жанрам, научных или новостных статей по темам, музыкальных композиций по инструментам и так далее.

В нашем случае мы будем работать с датасетом биотехнических новостей и классифицировать их по темам. Этот датасет уже предобработан: текст приведен к нижнему регистру, удалена пунктуация, все слова разделены проблелом.

In [3]:
# !pip3 install gensim numpy==1.26.4

In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score
import numpy as np
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, TensorDataset
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
from gensim.corpora import Dictionary

In [5]:
# nltk.download('wordnet')
# nltk.download('stopwords')
# nltk.download('punkt')
stopwords = nltk.corpus.stopwords.words("english")

In [6]:
dataset = pd.read_csv('/kaggle/input/dl-nlp-hw2/biotech_news.tsv', sep='\t')
dataset.sample(10)

,text,labels
2595,march 2 2021 prnewswire wugen inc a biotechnol...,"alliance & partnership, service & product prov..."
945,zibahub launches inclusive beauty to help cons...,"new initiatives & programs, product updates"
634,sketch the 24th edition of the china beauty ex...,"product launching & presentation, participatio..."
1459,naborforce reimagines care in richmond paige w...,"new initiatives & programs, executive statement"
202,a post shared by sara ali khan saraalikhan95 w...,other
2473,television disney star deploys new technologie...,product updates
1830,baltimore device startup regeltec gains moment...,product updates
555,june 21 2018 rarely in the scientific realm do...,m&a
546,tweet ibi group inc tse ibg analysts at pi fin...,company description
2877,2020 13 11 2831 by dennis niebel md sietske po...,article publication


## Предобработка лейблов


__Задание 1 (0.5 балла)__. Как вы можете заметить, лейблы записаны в виде строк, разделенных запятыми. Для работы с ними нам нужно преобразовать их в числа. Так как каждый объект может принадлежать нескольким классам, закодируйте лейблы в виде векторов из 0 и 1, где 1 означает, что объект принадлежит соответствующему классу, а 0 – не принадлежит. Имея такую кодировку, мы сможем обучить модель, решая задачу бинарной классификации для каждого класса.

In [7]:
# your code here
all_labels = sorted(list(set((', '.join(dataset['labels'])).split(', '))))
all_labels

['alliance & partnership',
 'article publication',
 'clinical trial sponsorship',
 'closing',
 'company description',
 'department establishment',
 'event organization',
 'executive appointment',
 'executive statement',
 'expanding geography',
 'expanding industry',
 'foundation',
 'funding round',
 'hiring',
 'investment in public company',
 'ipo exit',
 'm&a',
 'new initiatives & programs',
 'new initiatives or programs',
 'other',
 'participation in an event',
 'partnerships & alliances',
 'patent publication',
 'product launching & presentation',
 'product updates',
 'regulatory approval',
 'service & product providing',
 'subsidiary establishment',
 'support & philanthropy']

In [8]:
dataset['labels'] = list(map(lambda labels: np.isin(all_labels, labels.split(', ')).tolist() , dataset['labels']))
dataset['labels'] = list(map(lambda labels: np.asarray(labels, dtype=int), dataset['labels']))
dataset.sample(10)

,text,labels
594,alex nitkin waterton ceo david schwartz left a...,"[0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, ..."
157,state bank of southern utah date approved apri...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
258,v and based on the hulkcharacter by stan lee j...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
684,share by email todd owens the covid 19 pandemi...,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ..."
2583,devils lake journal grand forks altru health s...,"[0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, ..."
461,view comments chem trend a global leader in th...,"[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
1359,pinterest communication is key open those comm...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
2111,rashmi nair trainee reporter et now updated de...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, ..."
584,vitaly design a men s jewellery company charac...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1099,may 12 2021 11 52 am cdt updated may 12 2021 0...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


## Предобработка данных

В этом задании мы будем обучать рекуррентные нейронные сети. Как вы знаете, они работают лучше для коротких текстов, так как не очень хорошо улавливают далекие зависимости. Для уменьшение длин текстов их стоит почистить.

Сразу разделим выборку на обучающую и тестовую, чтобы считать все нужные статистики только по обучающей.

In [9]:
texts_train, texts_test, y_train, y_test = train_test_split(
    list(dataset['text']),
    list(dataset['labels']),
    test_size=0.2,  # do not change this
    random_state=0  # do not change this
)

__Задание 2 (1 балл)__. Удалите из текстов стоп слова, слишком редкие и слишком частые слова. Гиперпараметры подберите самостоятельно (в идеале их стоит подбирать по качеству на тестовой выборке). Если вы считаете, что стоит добавить еще какую-то обработку, то сделайте это. Важно не удалить ничего, что может повлиять на предсказание класса.

In [10]:
# your code here
def clear_texts(texts_train, texts_text, min_df=0.01, max_df=0.95):
    reg = re.compile(r'\w+')
    lemmatizer = WordNetLemmatizer()
    word_counts = {}

    clear_texts_train = []
    clear_texts_text = []

    for text in texts_train:
        text = reg.findall(text.lower())
        # лемматизация, убираем стоп-слова и числа
        clear_text = [lemmatizer.lemmatize(word) for word in text if (word not in stopwords) and (not word.isdigit())]
        clear_texts_train.append(clear_text)
        for word in clear_text:
            if word not in word_counts:
                word_counts[word] = 0 
            word_counts[word] += 1
    for text in texts_text:
        text = reg.findall(text.lower())
        # лемматизация, убираем стоп-слова и числа
        clear_text = [lemmatizer.lemmatize(word) for word in text if (word not in stopwords) and (not word.isdigit())]
        clear_texts_text.append(clear_text)
        for word in clear_text:
            if word not in word_counts:
                word_counts[word] = 0 
            word_counts[word] += 1
    
    # фильтруем по частоте
    counts = list(set(list(word_counts.values())))
    if min_df >= 1.0:
        min_df = int(min_df)
    else:
        min_df = int(np.quantile(counts, min_df))
    if max_df > 1.0:
        max_df = int(max_df)
    else:
        max_df = int(np.quantile(counts, max_df))
    for i, text in enumerate(clear_texts_train):
        text = [word for word in text if min_df <= word_counts[word] <= max_df]
        clear_texts_train[i] = text
    for i, text in enumerate(clear_texts_text):
        text = [word for word in text if min_df <= word_counts[word] <= max_df]
        clear_texts_text[i] = text
    return clear_texts_train, clear_texts_text


texts_train, texts_test = clear_texts(texts_train, texts_test, min_df=0.05, max_df=0.9)

In [11]:
print(texts_train[0])

['alex', 'waterton', 'ceo', 'david', 'left', 'pathway', 'living', 'ceo', 'pathway', 'living', 'facility', 'oak', 'hill', 'credit', 'pathway', 'living', 'real', 'estate', 'investment', 'firm', 'waterton', 'bought', 'majority', 'stake', 'pathway', 'living', 'chicago', 'senior', 'living', 'operator', 'facility', 'totaling', 'unit', 'waterton', 'bought', 'percent', 'half', 'seat', 'board', 'pathway', 'living', 'waterton', 'ceo', 'david', 'told', 'real', 'deal', 'chicago', 'investor', 'paid', 'sum', 'boost', 'percent', 'four', 'waterton', 'shared', 'ownership', 'housing', 'operator', 'helped', 'finance', 'acquisition', 'nine', 'facility', 'northwest', 'prospect', 'height', 'pathway', 'another', 'three', 'residence', 'pipeline', 'la', 'third', 'planned', 'area', 'step', 'east', 'coast', 'expansion', 'push', 'waterton', 'power', 'pathway', 'living', 'mix', 'independent', 'living', 'assisted', 'living', 'memory', 'facility', 'average', 'resident', 'age', 'expansion', 'put', 'prime', 'position'

__Задание 3 (1.5 балла)__. Осталось перевести тексты в индексы токенов, чтобы их можно было подавать в модель. У вас есть две опции, как это сделать:
1. __(+0 баллов)__ Токенизировать тексты по словам.
2. __(до +3 баллов)__ Реализовать свою токенизацию BPE. Количество баллов будет варьироваться в зависимости от эффективности реализации. При реализации нельзя пользоваться специализированными библиотеками.

Токенизируйте тексты, переведите их в списки индексов и сложите вместе с лейблами в `DataLoader`. Не забудьте добавить в `DataLoader` `collate_fn`, которая будет дополнять все короткие тексты в батче паддингами. Для маппинга токенов в индексы вам может пригодиться `gensim.corpora.dictionary.Dictionary`.

In [12]:
import itertools 
gensim_dict = Dictionary(texts_train + texts_test)

pad_idx = len(gensim_dict)

In [13]:
def encode_dataset(dataset):
    encoded = [gensim_dict.doc2idx(text) for text in dataset]
    return encoded

def my_pad(train, test):
    max_len = max([len(text) for text in train] + [len(text) for text in test])
    
    padded_train = [text + [pad_idx] * (max_len - len(text)) for text in train]
    padded_test = [text + [pad_idx] * (max_len - len(text)) for text in test]
    return torch.tensor(padded_train), torch.tensor(padded_test)

In [14]:
train_tokenized = encode_dataset(texts_train)
test_tokenized = encode_dataset(texts_test)

In [15]:
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

> Почему-то у меня collate_fn не применяется, поэтому делаю паддинг на этапе датасета

In [16]:
train_tokenized_pad, test_tokenized_pad = my_pad(train_tokenized, test_tokenized)

In [17]:
train_dataset = CustomDataset(train_tokenized_pad,  y_train)
test_dataset = CustomDataset(test_tokenized_pad, y_test)

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=512)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=512)

## Метрика качества

Перед тем, как приступить к обучению, нам нужно выбрать метрику оценки качества. Так как в задаче классификации с пересекающимися классами классы часто несбалансированы, чаще всего в качестве метрики берется [F1 score](https://en.wikipedia.org/wiki/F-score).

Функция `compute_f1` принимает истинные метки и предсказанные и считает среднее значение F1 по всем классам. Используйте ее для оценки качества моделей.

$$
F1_{total} = \frac{1}{K} \sum_{k=1}^K F1(Y_k, \hat{Y}_k),
$$
где $Y_k$ – истинные значения для класса k, а $\hat{Y}_k$ – предсказания.

In [18]:
from sklearn.metrics import f1_score

def compute_f1(y_true, y_pred):
    assert y_true.ndim == 2
    assert y_true.shape == y_pred.shape

    return f1_score(y_true, y_pred, average='macro')

## Обучение моделей

### RNN

В качестве бейзлайна обучим самую простую рекуррентную нейронную сеть. Напомним, что блок RNN выглядит таким образом.

<img src="https://i.postimg.cc/yYbNBm6G/tg-image-1635618906.png" alt="drawing" width="400"/>

Его скрытое состояние обновляется по формуле
$h_t = \sigma(W x_{t} + U h_{t-1} + b_h)$. А предсказание считается с помощью применения линейного слоя к последнему токену
$o_T = V h_T + b_o$. В качестве функции активации выберите гиперболический тангенс. 

__Задание 4 (2 балла)__. Реализуйте RNN в соответствии с формулой выше и обучите ее на нашу задачу. Нулевой скрытый вектор инициализируйте нулями, так модель будет обучаться стабильнее, чем при случайной инициализации. После этого замеряйте качество на тестовой выборке. У вас должно получиться значение F1 не меньше 0.33, а само обучение не должно занимать много времени.

In [19]:
import warnings
warnings.filterwarnings('ignore')

In [20]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, output_size, device):
        super(RNN, self).__init__()
        self.device = device
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx, device=device)
        
        self.W = nn.Linear(embedding_dim, hidden_size, device=device)
        self.U = nn.Linear(hidden_size, hidden_size, device=device)
        
        self.V = nn.Linear(hidden_size, output_size, device=device)

        # for layer in [self.W, self.U, self.V]:
        #     nn.init.xavier_uniform_(layer.weight)
        
    def forward(self, x):
        mask = (x != pad_idx).float()
        embedded = self.embedding(x)
        
        batch_size, seq_len, _ = embedded.shape
        h_prev = torch.zeros(batch_size, self.hidden_size, device=x.device)
        
        for t in range(seq_len):
            x_t = embedded[:, t, :]
            mask_t = mask[:, t].unsqueeze(1)
            h_t = torch.tanh(self.W(x_t) + self.U(h_prev))
            h_t = h_t * mask_t + h_prev * (1 - mask_t)
            h_prev = h_t
        
        output = self.V(h_t)
        return output

def train_model(model, train_loader, val_loader, epochs=10, lr=0.001, class_weights=None):
    criterion = nn.BCEWithLogitsLoss(pos_weight=class_weights)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for texts, labels in tqdm(train_loader):
            optimizer.zero_grad()
            outputs = model(texts.to(device))
            loss = criterion(outputs.cpu(), labels.float())
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
            optimizer.step()
            train_loss += loss.item()
        
        model.eval()
        val_preds, val_true = [], []
        with torch.no_grad():
            for texts, labels in val_loader:
                outputs = model(texts.to(device))
                preds = torch.sigmoid(outputs) > 0.5
                val_preds.extend(preds.cpu().numpy())
                val_true.extend(labels.cpu().numpy())
                
        accuracy = accuracy_score(val_true, val_preds)
        val_f1 = compute_f1(np.array(val_true, dtype=int), np.array(val_preds, dtype=int))
        print(f'Epoch {epoch+1}: Train Loss = {train_loss/len(train_loader):.4f}, \
                Val F1 = {val_f1:.4f}, Val acc = {accuracy:.4f}')


def evaluate_model(model, test_loader):
    model.eval()
    test_preds, test_true = [], []
    with torch.no_grad():
        for texts, labels in test_loader:
            outputs = model(texts.to(device))
            preds = torch.sigmoid(outputs) > 0.5
            test_preds.extend(preds.cpu().numpy())
            test_true.extend(labels.cpu().numpy())
    
    accuracy = accuracy_score(test_true, test_preds)
    f1 = compute_f1(np.array(test_true), np.array(test_preds))
    print(f'Test Accuracy: {accuracy:.4f}, Test F1: {f1:.4f}')


In [21]:
def calculate_class_weights(loader, extra_weight=1.0):
    labels = np.asarray([sample[1] for sample in loader.dataset])
    n_samples, n_classes = labels.shape
    
    positive_counts = np.sum(labels, axis=0)
    positive_counts[positive_counts == 0] = 1
    positive_counts[positive_counts == n_classes] = n_classes - 1
    negative_counts = n_samples - positive_counts
    
    # Веса для положительных классов (редких)
    pos_weights = extra_weight * negative_counts / positive_counts
    
    return torch.tensor(pos_weights)

In [22]:
vocab_size = len(gensim_dict.keys()) + 1
embedding_dim = 100
hidden_size = 256
output_size = len(all_labels)
class_weights = calculate_class_weights(train_loader, extra_weight=1.0)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [21]:
model = RNN(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    hidden_size=hidden_size,
    output_size=output_size,
    device=device
)
# print(list(model.parameters()))

# raise NotImplementedError
train_model(model.to(device), train_loader, test_loader,
            epochs=40, lr=0.01, class_weights=class_weights)

evaluate_model(model, test_loader)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1: Train Loss = 1.3051,                 Val F1 = 0.1116, Val acc = 0.0000


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 2: Train Loss = 0.9476,                 Val F1 = 0.1196, Val acc = 0.0066


  0%|          | 0/5 [00:00<?, ?it/s]

...

Epoch 38: Train Loss = 0.1452,                 Val F1 = 0.2364, Val acc = 0.1184


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 39: Train Loss = 0.1454,                 Val F1 = 0.2061, Val acc = 0.0987


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 40: Train Loss = 0.1363,                 Val F1 = 0.2549, Val acc = 0.1299
Test Accuracy: 0.1299, Test F1: 0.2549


> На 32 эпохе добились качества 0.33 по F1 score, потом выскочили из оптимума.

### LSTM

<img src="https://i.postimg.cc/pL5LdmpL/tg-image-2290675322.png" alt="drawing" width="400"/>

Теперь перейдем к более продвинутым рекурренным моделям, а именно LSTM. Из-за дополнительного вектора памяти эта модель должна гораздо лучше улавливать далекие зависимости, что должно напрямую отражаться на качестве.

Параметры блока LSTM обновляются вот так ($\sigma$ означает сигмоиду):
\begin{align}
f_{t} &= \sigma(W_f x_{t} + U_f h_{t-1} + b_f) \\ 
i_{t} &= \sigma(W_i x_{t} + U_i h_{t-1} + b_i) \\
\tilde{c}_{t} &= \tanh(W_c x_{t} + U_c h_{t-1} + b_i) \\
c_{t} &= f_t \odot c_{t-1} + i_t \odot \tilde{c}_t \\
o_{t} &= \sigma(W_t x_{t} + U_t h_{t-1} + b_t) \\
h_t &= o_t \odot \tanh(c_t)
\end{align}

__Задание 5 (2 балла).__ Реализуйте LSTM по описанной схеме. Выберите гиперпараметры LSTM так, чтобы их общее число (без учета слоя эмбеддингов) примерно совпадало с числом параметров обычной RNN, но размерность скрытого слоя была не меньше 64. Так мы будем сравнивать архитектуры максимально независимо. Обучите LSTM до сходимости и сравните качество с RNN на тестовой выборке. Удалось ли получить лучший результат? Как вы можете это объяснить?

> Каким ставить hidden_size ? 
> - В RNN параметров: $emb\_dim \times hid\_size$ на матрицу W, $hid\_size$ на сдвиг $b_h$, еще $hid\_size\times hid\_size$ на U, и еще классификатор V со сдвигом на $hid\_size \times out\_dim + out\_dim $ итого $hid\_size\times (hid\_size + emb\_dim + out\_dim + 1) + out\_dim$ параметров. В числах это $256 \times (256 + 100 + 29 + 1) + 29 = 98 845 $
> - В LSTM. На матрицы $W_*$: $4 \times emb\_dim \times hid\_size$, на сдвиги $b_*$ еще $4 \times hid\_size$, $U_*$ на $4 \times hid\_size \times hid\_size$ параметров, и классификатор на $hid\_size \times out\_dim + out\_dim $ параметров. Итого $hid\_size \times (4 hid\_size + 4 emb\_dim + out\_dim) + out\_dim$ $= x (4x + 400 + 29) + 29$. Чтобы параметров было как в RNN, $hid\_dim = x \approx 112$, возьмем скрытые слои размера $hidden\_size = 128$.

In [23]:
# your code here


class LSTMLayer(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size

        self.W_f = nn.Linear(input_size, hidden_size)
        self.W_i = nn.Linear(input_size, hidden_size)
        self.W_c = nn.Linear(input_size, hidden_size)
        self.W_t = nn.Linear(input_size, hidden_size)

        self.U_f = nn.Linear(hidden_size, hidden_size, bias=False)
        self.U_i = nn.Linear(hidden_size, hidden_size, bias=False)
        self.U_c = nn.Linear(hidden_size, hidden_size, bias=False)
        self.U_t = nn.Linear(hidden_size, hidden_size, bias=False)

    def forward(self, x, hc_prev, mask=None):
        h_prev, c_prev = hc_prev
        f_t = torch.sigmoid(self.W_f(x) + self.U_f(h_prev))
        i_t = torch.sigmoid(self.W_i(x) + self.U_i(h_prev))
        c_tilde = torch.tanh(self.W_c(x) + self.U_c(h_prev))
        c_t = f_t * c_prev + i_t * c_tilde
        o_t = torch.sigmoid(self.W_t(x) + self.U_t(h_prev))
        h_t = o_t * torch.tanh(c_t)

        if mask is not None:
            h_t = h_t * mask + h_prev * (1 - mask)
            c_t = c_t * mask + c_prev * (1 - mask)

        return h_t, c_t


class MultiLayerLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size

        self.layers = nn.ModuleList()
        for i in range(num_layers):
            layer_input_size = input_size if (i == 0) else hidden_size
            self.layers.append(LSTMLayer(input_size, hidden_size))

    def forward(self, x, mask=None):
        batch_size, seq_len, _ = x.shape

        h_t = torch.zeros(batch_size, self.hidden_size, device=x.device)
        c_t = torch.zeros(batch_size, self.hidden_size, device=x.device)

        for t in range(seq_len):
            x_t = x[:, t, :]
            mask_t = None
            if mask is not None: 
                mask_t = mask[:, t].unsqueeze(1)

            for layer in self.layers:
                h_t, c_t = layer(x_t, (h_t, c_t), mask=mask_t)

        return h_t


class MultiLabelLSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, output_size, num_layers=1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.lstm = MultiLayerLSTM(embedding_dim, hidden_size, num_layers)
        self.classifier = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # x shape: (batch_size, seq_len)
        mask = (x != pad_idx).float()
        x = self.embedding(x)  # (batch_size, seq_len, embedding_dim)

        h_T = self.lstm(x, mask=mask)
        output = self.classifier(h_T)

        return output


In [23]:
model = MultiLabelLSTMClassifier(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    hidden_size=hidden_size // 2,
    output_size=output_size
)
# print(list(model.parameters()))

# raise NotImplementedError
train_model(model.to(device), train_loader, test_loader,
            epochs=35, lr=0.005, class_weights=class_weights)

evaluate_model(model, test_loader)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1: Train Loss = 1.2893,                 Val F1 = 0.1052, Val acc = 0.0000


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 2: Train Loss = 1.1926,                 Val F1 = 0.1114, Val acc = 0.0000


  0%|          | 0/5 [00:00<?, ?it/s]

...

Epoch 33: Train Loss = 0.0276,                 Val F1 = 0.3404, Val acc = 0.2812


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 34: Train Loss = 0.0258,                 Val F1 = 0.3480, Val acc = 0.2845


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 35: Train Loss = 0.0237,                 Val F1 = 0.3453, Val acc = 0.2862
Test Accuracy: 0.2862, Test F1: 0.3453


> Стало чуть лучше, уверенно достигли F1 score = 0.34. Эта модель сложнее, у нее больше потенциал, за счет дополнительных векторов взаимодействия внутри модели улавливается больше связей. Но наверное ее ограничивает относительно небольшое число параметров, ведь мы постарались сделать их примерно как в RNN.

__Задание 6 (2 балла).__ Главный недостаток RNN моделей заключается в том, что при сжатии всей информации в один вектор, важные детали пропадают. Для решения этой проблемы был придуман механизм внимания. Реализуйте его по [оригинальной статье](https://arxiv.org/abs/1409.0473). Замерьте качество и сделайте выводы.   
Обратите внимание, что метод был предложен для Encoder-Decoder моделей. В нашем случае декодера нет, поэтому встройте внимание в энкодер: каждый блок LSTM будет смотреть на выходы всех предыдущих блоков.   

In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Attention(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.W_a = nn.Linear(hidden_size, hidden_size, bias=False)
        self.U_a = nn.Linear(hidden_size, hidden_size, bias=False)
        self.v_a = nn.Linear(hidden_size, 1, bias=False)

    def forward(self, hidden, encoder_outputs, mask=None):
        hidden_expanded = hidden.unsqueeze(1)
        energies = self.v_a(torch.tanh(self.W_a(encoder_outputs) + self.U_a(hidden_expanded))).squeeze(-1)  # (batch_size, seq_len)
        
        if mask is not None:
            energies[mask == 0] = -1e9
        
        attention_weights = F.softmax(energies, dim=-1)
        
        # Вычисляем контекстный вектор
        context = torch.bmm(attention_weights.unsqueeze(1), encoder_outputs)
        context = context.squeeze(1) 
        
        return context


class LSTMLayerWithAttention(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size

        # Стандартные веса LSTM
        self.W_f = nn.Linear(input_size, hidden_size)
        self.W_i = nn.Linear(input_size, hidden_size)
        self.W_c = nn.Linear(input_size, hidden_size)
        self.W_o = nn.Linear(input_size, hidden_size)

        self.U_f = nn.Linear(hidden_size, hidden_size, bias=False)
        self.U_i = nn.Linear(hidden_size, hidden_size, bias=False)
        self.U_c = nn.Linear(hidden_size, hidden_size, bias=False)
        self.U_o = nn.Linear(hidden_size, hidden_size, bias=False)

        # Веса для механизма внимания
        self.attention = Attention(hidden_size)
        
        # Дополнительные веса для объединения контекста с входом
        self.W_ctx = nn.Linear(hidden_size, input_size)

    def forward(self, x, hc_prev, all_previous_outputs, mask=None):
        h_prev, c_prev = hc_prev

        # Вычисляем контекст через внимание
        context = self.attention(h_prev, all_previous_outputs, mask)
        
        # Объединяем контекст с входным вектором
        x_enhanced = x + self.W_ctx(context)

        # Стандартные вычисления LSTM с улучшенным входом
        f_t = torch.sigmoid(self.W_f(x_enhanced) + self.U_f(h_prev))
        i_t = torch.sigmoid(self.W_i(x_enhanced) + self.U_i(h_prev))
        c_hat_t = torch.tanh(self.W_c(x_enhanced) + self.U_c(h_prev))
        c_t = f_t * c_prev + i_t * c_hat_t
        o_t = torch.sigmoid(self.W_o(x_enhanced) + self.U_o(h_prev))
        h_t = o_t * torch.tanh(c_t)

        if mask is not None:
            # Применяем маску только если она предоставлена
            mask_expanded = mask.unsqueeze(-1)
            h_t = h_t * mask_expanded + h_prev * (1 - mask_expanded)
            c_t = c_t * mask_expanded + c_prev * (1 - mask_expanded)

        return h_t, c_t


class MultiLayerLSTMWithAttention(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size

        self.layers = nn.ModuleList()
        for i in range(num_layers):
            layer_input_size = input_size if (i == 0) else hidden_size
            self.layers.append(LSTMLayerWithAttention(layer_input_size, hidden_size))

    def forward(self, x, mask=None):
        batch_size, seq_len, _ = x.shape

        # Инициализируем скрытые состояния
        h_t = torch.zeros(batch_size, self.hidden_size, device=x.device)
        c_t = torch.zeros(batch_size, self.hidden_size, device=x.device)
        
        # Тензор для хранения всех выходов текущего слоя
        all_layer_outputs = torch.zeros(batch_size, seq_len, self.hidden_size, device=x.device)

        for t in range(seq_len):
            x_t = x[:, t, :]

            mask_t = None
            if mask is not None:
                mask_t = mask[:, t]

            # Получаем все предыдущие выходы до текущего временного шага
            if t == 0:
                previous_outputs = torch.zeros(batch_size, 1, self.hidden_size, device=x.device)
            else:
                previous_outputs = all_layer_outputs[:, :t, :]

            for layer_idx, layer in enumerate(self.layers):
                h_t, c_t = layer(x_t, (h_t, c_t), previous_outputs, mask_t)
                
                all_layer_outputs[:, t, :] = h_t

        return h_t


class MultiLabelLSTMClassifierWithAttention(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, output_size, num_layers=1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.lstm = MultiLayerLSTMWithAttention(embedding_dim, hidden_size, num_layers)
        self.classifier = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # x shape: (batch_size, seq_len)
        mask = (x != pad_idx).float()
        x_emb = self.embedding(x)

        h_T = self.lstm(x_emb, mask=mask)
        output = self.classifier(h_T)

        return output

In [25]:
model = MultiLabelLSTMClassifierWithAttention(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    hidden_size=hidden_size // 2,
    output_size=output_size
)
# print(list(model.parameters()))

# raise NotImplementedError
train_model(model.to(device), train_loader, test_loader,
            epochs=35, lr=0.005, class_weights=class_weights)

evaluate_model(model, test_loader)

  0%|          | 0/5 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 60.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 24.12 MiB is free. Process 4359 has 14.71 GiB memory in use. Of the allocated memory 14.28 GiB is allocated by PyTorch, and 320.54 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

> Написать-то я написала модель.. только что-то тут с памятью не так: видимо где-то я сохраняю лишие матрицы, потому что по логике все как надо.

__Задание 7 (1 балл).__ Добавьте в вашу реализации возможность увеличивать число слоев LSTM. Обучите модель с двумя слоями и замерьте качество. Сделайте выводы: стоит ли увеличивать размер модели?

In [41]:
# your code here

model = MultiLabelLSTMClassifier(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    hidden_size=hidden_size // 2,
    output_size=output_size,
    num_layers=2
)
# print(list(model.parameters()))

# raise NotImplementedError
train_model(model.to(device), train_loader, test_loader,
            epochs=35, lr=0.005, class_weights=class_weights)

evaluate_model(model, test_loader)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1: Train Loss = 1.2980,                 Val F1 = 0.1025, Val acc = 0.0000


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 2: Train Loss = 1.1800,                 Val F1 = 0.1093, Val acc = 0.0000


  0%|          | 0/5 [00:00<?, ?it/s]

...

Epoch 33: Train Loss = 0.0258,                 Val F1 = 0.3359, Val acc = 0.3026


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 34: Train Loss = 0.0238,                 Val F1 = 0.3262, Val acc = 0.3026


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 35: Train Loss = 0.0226,                 Val F1 = 0.3288, Val acc = 0.2993
Test Accuracy: 0.2993, Test F1: 0.3288


Качество модели с 2 слоями незначительно выросло. Она получше, потому что в ней больше параметров, и она может выучить более сложные взаимосвязи между токенами. С другой стороны, ей может не хватает контекста, т.е. она не способна учить далекие связи и закономерности.